## <font color='darkblue'>Introduction</font>
([article source](https://machinelearningmastery.com/stacking-ensemble-machine-learning-with-python/)) **Stacking or Stacked Generalization is an ensemble machine learning algorithm. It uses a meta-learning algorithm to learn how to best combine the predictions from two or more base machine learning algorithms.**

The benefit of stacking is that it can harness the capabilities of a range of well-performing models on a classification or regression task and make predictions that have better performance than any single model in the ensemble.

In this tutorial, you will discover the stacked generalization ensemble or stacking in Python. After completing this tutorial, you will know:
* Stacking is an ensemble machine learning algorithm that learns how to best combine the predictions from multiple well-performing machine learning models.
* The scikit-learn library provides a standard implementation of the stacking ensemble in Python.
* How to use stacking ensembles for regression and classification predictive modeling.

This tutorial is divided into four parts; they are:
* [**Stacked Generalization**](#sect1)
* [**Stacking Scikit-Learn API**](#sect2)
* [**Stacking for Classification**](#sect3)
* **Stacking for Regression**

## <font color='darkblue'>Stacked Generalization</font>
Stacked Generalization or “Stacking” for short is an ensemble machine learning algorithm. It **involves combining the predictions from multiple machine learning models on the same dataset, like bagging and boosting**. Stacking addresses the question:
> Given multiple machine learning models that are skillful on a problem, but in different ways, how do you choose which model to use (trust)?

**The approach to this question is to use another machine learning model that learns when to use or trust each model in the ensemble.**
* **Unlike bagging**, in stacking, the models are typically different (<font color='brown'>e.g. not all decision trees</font>) and fit on the same dataset (<font color='brown'>e.g. instead of samples of the training dataset</font>).
* **Unlike boosting**, in stacking, a single model is used to learn how to best combine the predictions from the contributing models (<font color='brown'>e.g. instead of a sequence of models that correct the predictions of prior models</font>).

The architecture of a stacking model involves two or more base models, often referred to as level-0 models, and a meta-model that combines the predictions of the base models, referred to as a level-1 model:
* **Level-0 Models** (<font color='brown'>Base-Models</font>): Models fit on the training data and whose predictions are compiled.
* **Level-1 Model** (<font color='brown'>Meta-Model</font>): Model that learns how to best combine the predictions of the base models.

**The meta-model is trained on the predictions made by base models on out-of-sample data.** That is, data not used to train the base models is fed to the base models, predictions are made, and these predictions, along with the expected outputs, provide the input and output pairs of the training dataset used to fit the meta-model.

The outputs from the base models used as input to the meta-model may be real value in the case of regression, and probability values, probability like values, or class labels in the case of classification.

The most common approach to preparing the training dataset for the meta-model is via [k-fold cross-validation](https://machinelearningmastery.com/k-fold-cross-validation/) of the base models, where the [out-of-fold predictions](https://machinelearningmastery.com/out-of-fold-predictions-in-machine-learning/) are used as the basis for the training dataset for the meta-model.

**The training data for the meta-model may also include the inputs to the base models**, e.g. input elements of the training data. This can provide an additional context to the meta-model as to how to best combine the predictions from the meta-model.

Once the training dataset is prepared for the meta-model, the meta-model can be trained in isolation on this dataset, and the base-models can be trained on the entire original training dataset.

**Stacking is appropriate when multiple different machine learning models have skill on a dataset, but have skill in different ways.** Another way to say this is that the predictions made by the models or the errors in predictions made by the models are uncorrelated or have a low correlation.

**Base-models are often complex and diverse. As such, it is often a good idea to use a range of models that make very different assumptions about how to solve the predictive modeling task**, such as linear models, decision trees, support vector machines, neural networks, and more. Other ensemble algorithms may also be used as base-models, such as random forests.

**The meta-model is often simple, providing a smooth interpretation of the predictions made by the base models**. As such, linear models are often used as the meta-model, such as linear regression for regression tasks (<font color='brown'>predicting a numeric value</font>) and logistic regression for classification tasks (<font color='brown'>predicting a class label</font>). Although this is common, it is not required.

The use of a simple linear model as the meta-model often gives stacking the colloquial name “blending.” As in the prediction is a weighted average or blending of the predictions made by the base models.

The [**super learner**](https://machinelearningmastery.com/super-learner-ensemble-in-python/) may be considered a specialized type of stacking.

**Stacking is designed to improve modeling performance, although is not guaranteed to result in an improvement in all cases.**

Achieving an improvement in performance depends on the complexity of the problem and whether it is sufficiently well represented by the training data and complex enough that there is more to learn by combining predictions. **It is also dependent upon the choice of base models and whether they are sufficiently skillful and sufficiently uncorrelated in their predictions** (<font color='brown'>or errors</font>).

If a base-model performs as well as or better than the stacking ensemble, the base model should be used instead, given its lower complexity (<font color='brown'>e.g. it’s simpler to describe, train and maintain</font>).

<a id='sect2'></a>
## <font color='darkblue'>Stacking Scikit-Learn API</font>
Stacking can be implemented from scratch, although this can be challenging for beginners. For an example of implementing stacking from scratch in Python, see the tutorial:
* [**How to Implement Stacked Generalization (Stacking) From Scratch With Python**](https://machinelearningmastery.com/implementing-stacking-scratch-python/)

For an example of implementing stacking from scratch for deep learning, see the tutorial:
* [**How to Develop a Stacking Ensemble for Deep Learning Neural Networks in Python**](https://machinelearningmastery.com/stacking-ensemble-for-deep-learning-neural-networks/)

The scikit-learn Python machine learning library provides an implementation of stacking for machine learning. It is **available in version 0.22 of the library and higher**. First, confirm that you are using a modern version of the library by running the following script:

In [1]:
# check scikit-learn version
import sklearn
print(sklearn.__version__)

0.23.2


Stacking is provided via the [**StackingRegressor**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html) and [**StackingClassifier**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html) classes.

Both models operate the same way and take the same arguments. Using the model requires that you specify a list of estimators (<font color='brown'>level-0 models</font>), and a final estimator (<font color='brown'>level-1 or meta-model</font>).

**A list of level-0 models or base models is provided via the “<font color='violet'>estimators</font>” argument.** This is a Python list where each element in the list is a tuple with the name of the model and the configured model instance.

For example, below defines two level-0 models:
```python
models = [('lr',LogisticRegression()),('svm',SVC()), ...]
stacking = StackingClassifier(estimators=models)
```

Each model in the list may also be a [**Pipeline**](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html), including any data preparation required by the model prior to fitting the model on the training dataset. For example:
```python
models = [('lr',LogisticRegression()),('svm',make_pipeline(StandardScaler(),SVC())), ...]
stacking = StackingClassifier(estimators=models)
```

The level-1 model or meta-model is provided via the “<font color='violet'>final_estimator</font>” argument. By default, this is set to [**LinearRegression**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) for regression and [**LogisticRegression**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) for classification, and these are sensible defaults that you probably do not want to change.

**The dataset for the meta-model is prepared using cross-validation. By default, 5-fold cross-validation is used**, although this can be changed via the “<font color='violet'>cv</font>” argument and set to either a number (<font color='brown'>e.g. 10 for 10-fold cross-validation</font>) or a cross-validation object (<font color='brown'>e.g. [**StratifiedKFold**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html)</font>).

Sometimes, **better performance can be achieved if the dataset prepared for the meta-model also includes inputs to the level-0 models,** e.g. the input training data. This can be achieved by setting the “<font color='violet'>passthrough</font>” argument to True and is not enabled by default.

Now that we are familiar with the stacking API in scikit-learn, let’s look at some worked examples.

<a id='sect3'></a>
## <font color='darkblue'>Stacking for Classification</font>
**In this section, we will look at using stacking for a classification problem.**

First, we can use the [make_classification()](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html) function to create a synthetic binary classification problem with 1,000 examples and 20 input features. The complete example is listed below.

In [1]:
# test classification dataset
from sklearn.datasets import make_classification
# define dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# summarize the dataset
print(X.shape, y.shape)

(1000, 20) (1000,)


Next, we can evaluate a suite of different machine learning models on the dataset. Specifically, we will evaluate the following five algorithms:
* Logistic Regression.
* k-Nearest Neighbors.
* Decision Tree.
* Support Vector Machine.
* Naive Bayes.

**Each algorithm will be evaluated using default model hyperparameters**. The function <font color='blue'>get_models()</font> below creates the models we wish to evaluate.

In [2]:
# get a list of models to evaluate
def get_models():
    models = dict()
    models['lr'] = LogisticRegression()
    models['knn'] = KNeighborsClassifier()
    models['cart'] = DecisionTreeClassifier()
    models['svm'] = SVC()
    models['bayes'] = GaussianNB()
    return models

Each model will be evaluated using repeated k-fold cross-validation. The <font color='blue'>evaluate_model()</font> function below takes a model instance and returns a list of scores from three repeats of stratified 10-fold cross-validation (Check [**RepeatedStratifiedKFold**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedStratifiedKFold.html#sklearn.model_selection.RepeatedStratifiedKFold)).

In [3]:
# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return scores

We can then report the mean performance of each algorithm and also create a box and whisker plot to compare the distribution of accuracy scores for each algorithm. Tying this together, the complete example is listed below.

## <font color='darkblue'>Supplement</font>
* [機器學習: Ensemble learning之Bagging、Boosting和AdaBoost](https://medium.com/@chih.sheng.huang821/%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92-ensemble-learning%E4%B9%8Bbagging-boosting%E5%92%8Cadaboost-af031229ebc3)